In [2]:
from mmseg.apis import inference_model, init_model, show_result_pyplot
import mmcv

config_file = 'pspnet_r50-d8_4xb2-40k_cityscapes-512x1024.py'
checkpoint_file = 'pspnet_r50-d8_512x1024_40k_cityscapes_20200605_003338-2966598c.pth'

# build the model from a config file and a checkpoint file
model = init_model(config_file, checkpoint_file, device='cuda:0')

# test a single image and show the results
img = 'demo/demo.png'  # or img = mmcv.imread(img), which will only load it once
result = inference_model(model, img)
# visualize the results in a new window
show_result_pyplot(model, img, result, show=True)
# or save the visualization results to image files
# you can change the opacity of the painted segmentation map in (0, 1].
show_result_pyplot(model, img, result, show=True, out_file='result.jpg', opacity=0.5)
# test a video and show the results
video = mmcv.VideoReader('video.mp4')
for frame in video:
   result = inference_model(model, frame)
   show_result_pyplot(model, frame, result, wait_time=1)

c:\Users\Pasca\anaconda3\envs\depth\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


FileNotFoundError: [Errno 2] No such file or directory: 'pspnet_r50-d8_4xb2-40k_cityscapes-512x1024.py'

In [41]:
import os
import numpy as np
from PIL import Image
from mmengine.dataset import BaseDataset

class CustomDataset(BaseDataset):
    # CLASSES = ('class1', 'class2', 'class3', ...)  # Define your classes here
    # PALETTE = ...  # Define your color palette for visualization here

    def __init__(self, data_root, img_dir, ann_dir, split, img_suffix='.png', ann_suffix='.png', **kwargs):
        self.data_root = data_root
        self.img_dir = img_dir
        self.ann_dir = ann_dir
        self.split = split
        self.img_suffix = img_suffix
        self.ann_suffix = ann_suffix
        self.data_infos = self.load_annotations(self.data_root, self.img_dir, self.ann_dir, self.split)
        print(f'len2: {len(self.data_infos)}')  # Debug print
        super().__init__(**kwargs)
        print(f'len3: {len(self.data_infos)}')  # Debug print

    def load_annotations(self, data_root, img_dir, ann_dir, split):
        img_folder = os.path.join(data_root, img_dir)
        ann_folder = os.path.join(data_root, ann_dir)
        split_file = os.path.join(data_root, f'{split}.txt')

        print(f'img_folder: {img_folder}')  # Debug print
        print(f'ann_folder: {ann_folder}')  # Debug print
        print(f'split_file: {split_file}')  # Debug print

        data_infos = []
        with open(split_file, 'r') as f:
            lines = f.readlines()
            for line in lines:
                img, ann = line.strip().split(' ')
                data_infos.append({
                    'filename': os.path.join(img_folder, img),
                    'ann_info': {
                        'seg_map': os.path.join(ann_folder, ann)
                    }
                })

        return data_infos
    
    def load_data_list(self):
        print("load_data_list in CustomDataset is called")  # Debug print
        print(f'len1: {len(self.data_infos)}')  # Debug print
        return self.data_infos
    
    def filter_data(self):
        print("filter_data in CustomDataset is called")
        print(f'len5: {len(self.data_infos)}')  # Debug print
        return self.data_infos

    def get_ann_info(self, idx):
        return self.data_infos[idx]['ann_info']

    def __getitem__(self, idx):
        img = Image.open(self.data_infos[idx]['filename'])
        ann_info = self.get_ann_info(idx)
        seg_map = Image.open(ann_info['seg_map'])
        return img, seg_map

    def __len__(self):
        return len(self.data_infos)

In [43]:
# code\UsefullnessOfDepth\datasets\SynthDet_light_angle_distribution

# build the dataset
dataset_type = 'CustomDataset'
data_root = '..\datasets\SynthDet_light_angle_distribution'
img_dir = 'RGB'
ann_dir = 'labels'
split = 'train'
pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations'),
    dict(type='PackSegInputs')
]

dataset = CustomDataset(data_root, img_dir, ann_dir, split, pipeline=pipeline)

print(dataset.metainfo)

# Print the length of the dataset
print(f'Number of samples: {len(dataset)}')

# Access and print the first item
img, seg_map = dataset[0]
print(f'First image shape: {np.array(img).shape}')
print(f'First segmentation map shape: {np.array(seg_map).shape}')

# Access and print the last item
img, seg_map = dataset[-1]
print(f'Last image shape: {np.array(img).shape}')
print(f'Last segmentation map shape: {np.array(seg_map).shape}')

img_folder: ..\datasets\SynthDet_light_angle_distribution\RGB
ann_folder: ..\datasets\SynthDet_light_angle_distribution\labels
split_file: ..\datasets\SynthDet_light_angle_distribution\train.txt
len2: 1000


KeyError: 'LoadImageFromFile is not in the mmengine::transform registry. Please check whether the value of `LoadImageFromFile` is correct or it was registered as expected. More details can be found at https://mmengine.readthedocs.io/en/latest/advanced_tutorials/config.html#import-the-custom-module'